## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Porto Novo,BJ,6.4965,2.6036,82.22,81,87,6.53,overcast clouds
1,1,Knysna,ZA,-34.0363,23.0471,56.79,82,18,6.35,few clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,44.71,81,40,6.91,scattered clouds
3,3,Vaini,TO,-21.2000,-175.2000,84.36,79,40,6.91,scattered clouds
4,4,Kapaa,US,22.0752,-159.3190,82.38,84,95,5.01,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature (F) you would like for your vacation destination? '))
max_temp = float(input('What is the maximum temperature (F) you would like for your vacation destination? '))

What is the minimum temperature (F) you would like for your vacation destination? 75
What is the maximum temperature (F) you would like for your vacation destination? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Porto Novo,BJ,6.4965,2.6036,82.22,81,87,6.53,overcast clouds
3,3,Vaini,TO,-21.2000,-175.2000,84.36,79,40,6.91,scattered clouds
4,4,Kapaa,US,22.0752,-159.3190,82.38,84,95,5.01,moderate rain
5,5,Buraydah,SA,26.3260,43.9750,75.65,19,0,9.33,clear sky
9,9,Cabo San Lucas,MX,22.8909,-109.9124,86.61,42,100,10.27,overcast clouds
10,10,Bathsheba,BB,13.2167,-59.5167,81.21,69,40,16.11,scattered clouds
14,14,Grand Gaube,MU,-20.0064,57.6608,79.20,86,19,5.99,few clouds
17,17,Pran Buri,TH,12.3847,99.9098,77.76,82,56,3.22,broken clouds
18,18,Souillac,MU,-20.5167,57.5167,79.18,83,75,8.05,broken clouds
26,26,Rikitea,PF,-23.1203,-134.9692,78.49,70,7,13.67,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.replace("", np.NaN).dropna()
clean_df.count()

City_ID                215
City                   215
Country                215
Lat                    215
Lng                    215
Max Temp               215
Humidity               215
Cloudiness             215
Wind Speed             215
Current Description    215
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.tail(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
654,Doha,QA,80.58,clear sky,25.2867,51.5333,
657,Winslow,US,77.18,clear sky,35.0242,-110.6974,
658,Thinadhoo,MV,83.08,few clouds,0.5333,72.9333,
661,Lhokseumawe,ID,79.34,overcast clouds,5.1801,97.1507,
664,Bandarbeyla,SO,80.85,scattered clouds,9.4942,50.8122,
669,Kralendijk,BQ,79.03,clear sky,12.1500,-68.2667,
671,Ikom,NG,77.92,broken clouds,5.9667,8.7063,
672,Lucea,JM,77.86,light rain,18.4510,-78.1736,
673,Phangnga,TH,77.04,broken clouds,8.4509,98.5298,
679,Garowe,SO,77.50,scattered clouds,8.4054,48.4845,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

#Iterates through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat}, {lng}'
    
    #Sets up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    #Makes request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    #Gets the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    
    except:
        print('Hotel not found... Skipping')

Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name']!= '']

clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Porto Novo,BJ,82.22,overcast clouds,6.4965,2.6036,Tour Eiffel Hotel Benin
3,Vaini,TO,84.36,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
4,Kapaa,US,82.38,moderate rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
5,Buraydah,SA,75.65,clear sky,26.3260,43.9750,Mövenpick Qassim
9,Cabo San Lucas,MX,86.61,overcast clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
10,Bathsheba,BB,81.21,scattered clouds,13.2167,-59.5167,Atlantis Hotel
14,Grand Gaube,MU,79.20,few clouds,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
17,Pran Buri,TH,77.76,broken clouds,12.3847,99.9098,Waterside Resort
18,Souillac,MU,79.18,broken clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
26,Rikitea,PF,78.49,light rain,-23.1203,-134.9692,People ThankYou


In [10]:
# 8a. Create the output File (CSV)
output_data_file = '../Vacation_Search/WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>

<dt>City</dt><dd>{City}</dd>

<dt>Country</dt><dd>{Country}</dd>

<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>


</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))